# Testing global embedding function

In [2]:
from collections import deque, defaultdict
import torch

In [ ]:
def node_embedding_update_sum(start_node, ccn, k):
    embeddings_required = []
    dq = deque([(start_node, k, {start_node})])
    while dq:
        node, hop, nodes_visited = dq.popleft()
        embeddings_required.append([node, hop])
        if hop > 1 and node == start_node:
            embeddings_required += [[node, 0]]  * len(ccn[node]) # count the times 1-hop ccn visits itself
        elif hop > 1:
            embeddings_required += [[node, 0]] # add 0-hop whenever it is visited

        for neigh in ccn[node]:
            if neigh not in nodes_visited and hop>0:
                dq.append((neigh, hop-1, nodes_visited|{neigh}))

    return embeddings_required

def get_global_embedding(embeddings, ccn, node_client_map):
    hop_embeddings = []
    for hop in range(3):
        hop_matrix = []
        for node in range(len(node_client_map)):
            node_embdedding_sum = node_embedding_update_sum(node, ccn, hop)
            final_embedding = torch.zeros(embeddings[0][0][0].shape)
            for update_node, k in node_embdedding_sum:
                final_embedding += embeddings[node_client_map[update_node]][k][update_node]
            hop_matrix.append(final_embedding)
        hop_embeddings.append(hop_matrix)

    return hop_embeddings

In [ ]:
'''Create test graph 1'''
ccn_1 = defaultdict(list)
ccn_1[3].append(4)
ccn_1[4].append(3)

node_assignment_1 = [0,0,0,0,1,1,1,1]
embeddings_1 = torch.tensor([[[[1,1],[2,2],[3,3],[4,4],[0,0],[0,0],[0,0],[0,0]],
                 [[2,2],[4,4],[6,6],[8,8],[0,0],[0,0],[0,0],[0,0]],
                 [[3,3],[5,5],[7,7],[9,9],[0,0],[0,0],[0,0],[0,0]]],
                [[[0,0],[0,0],[0,0],[0,0],[1,1],[2,2],[3,3],[4,4]],
                [[0,0],[0,0],[0,0],[0,0],[2,2],[4,4],[6,6],[8,8]],
                [[0,0],[0,0],[0,0],[0,0],[3,3],[5,5],[7,7],[9,9]]]])

# node_embedding_update_sum(4, ccn_1, 0)
# get_global_embedding(embeddings_1, ccn_1, node_assignment_1)

# '''Create test graph 2'''
# ccn_2 = defaultdict(list)
# ccn_2[3].append(4)
# ccn_2[4].append(3)
# ccn_2[3].append(8)
# ccn_2[8].append(3)
# node_assignment_2 = [0,0,0,0,1,1,1,1,2,2,2]
# embeddings_1 = torch.tensor([[[[1,1],[2,2],[3,3],[4,4],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]],
#                  [[2,2],[4,4],[6,6],[8,8],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]],
#                  [[3,3],[5,5],[7,7],[9,9],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]],

#                 [[[0,0],[0,0],[0,0],[0,0],[1,1],[2,2],[3,3],[4,4],[0,0],[0,0],[0,0]],
#                 [[0,0],[0,0],[0,0],[0,0],[2,2],[4,4],[6,6],[8,8],[0,0],[0,0],[0,0]],
#                 [[0,0],[0,0],[0,0],[0,0],[3,3],[5,5],[7,7],[9,9],[0,0],[0,0],[0,0]]],

#                 [[[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[1,1],[2,2],[3,3]],
#                 [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[2,2],[4,4],[6,6]],
#                 [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[3,3],[5,5],[7,7]]],
#                 ])

# node_embedding_update_sum(3, ccn_2, 1)

In [ ]:
print(ccn_1)



defaultdict(<class 'list'>, {3: [4], 4: [3]})
